In [12]:
import os

import pandas as pd
import numpy as np
import torch

import bitsandbytes
import accelerate
from datasets import Dataset
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import Trainer, TrainingArguments



In [13]:
model_name = "01-ai/Yi-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
	 model_name,
	 device_map='auto',
	 quantization_config=transformers.BitsAndBytesConfig(),
	 torch_dtype=torch.bfloat16,
	 trust_remote_code=True 
).eval()

loading file tokenizer.model from cache at /Users/kwanshunquinsonhon/.cache/huggingface/hub/models--01-ai--Yi-6B/snapshots/58332cd8ca449a6a28a424dde80d03321ab03b41/tokenizer.model
loading file tokenizer.json from cache at /Users/kwanshunquinsonhon/.cache/huggingface/hub/models--01-ai--Yi-6B/snapshots/58332cd8ca449a6a28a424dde80d03321ab03b41/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /Users/kwanshunquinsonhon/.cache/huggingface/hub/models--01-ai--Yi-6B/snapshots/58332cd8ca449a6a28a424dde80d03321ab03b41/tokenizer_config.json
loading configuration file config.json from cache at /Users/kwanshunquinsonhon/.cache/huggingface/hub/models--01-ai--Yi-6B/snapshots/58332cd8ca449a6a28a424dde80d03321ab03b41/config.json
Model config LlamaConfig {
  "_name_or_path": "01-ai/Yi-6B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout"

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`